Kendall W über die Dimensionen


# =====================================================================
# 0. Setup: Pakete und Daten (HUMAN)
# =====================================================================

In [1]:
# =====================================================================
# Pakete
# =====================================================================
library(dplyr)
library(tidyr)
library(ggplot2)
library(reshape2)
library(knitr)
library(irr)
library(readxl)
library(xtable)


# =====================================================================
# 0. Daten einlesen
# =====================================================================
setwd("/Users/joern.ahlert/Desktop/Bachelorarbeit")

df_raw <- read_excel(
  path = "technische Umsetzung/Diagramme/Plausibilitätsbewertung von LLM Antworten2.xlsx"
)

# Beispiele:
# exclude_rows <- NULL        # Keine Zeile weglassen
# exclude_rows <- 3           # Nur Zeile 3 weglassen
# exclude_rows <- c(1, 5, 7)  # Zeilen 1, 5 und 7 weglassen

exclude_rows <- NULL  # <- HIER EINGEBEN, welche Zeile(n) weggelassen werden sollen

# Zeilen entfernen, falls angegeben
if (!is.null(exclude_rows)) {
  cat("Weglassen von Zeile(n):", exclude_rows, "\n")
  cat("Anzahl Zeilen vorher:", nrow(df_raw), "\n")
  df_raw <- df_raw[-exclude_rows, ]
  cat("Anzahl Zeilen nachher:", nrow(df_raw), "\n\n")
} else {
  cat("Keine Zeilen werden weggelassen.\n\n")
}


View(df_raw)
# =====================================================================
# 1. Antworttexte in numerische Scores umkodieren
# =====================================================================
likert_map <- c(
  "Trifft überhaupt nicht zu" = 1,
  "Trifft eher nicht zu"      = 2,
  "Trifft teilweise zu"       = 3,
  "Trifft eher zu"            = 4,
  "Trifft voll und ganz zu"   = 5
)

plaus_map <- c(
  "Unmöglich – Die Antwort widerspricht der Logik oder dem Kontext vollständig" = 1,
  "Technisch möglich – Theoretisch denkbar, aber extrem unwahrscheinlich"      = 2,
  "Plausibel – Könnte zutreffen, ist aber nicht der Normalfall"                = 3,
  "Wahrscheinlich – Klingt vernünftig und entspricht der Erwartung"            = 4,
  "Sehr wahrscheinlich – Sehr überzeugend; so würde ein Experte antworten"     = 5
)


meta_cols <- c("ID", "Startzeit", "Fertigstellungszeit",
               "E-Mail", "Name", "Zeitpunkt der letzten Änderung")
item_cols <- setdiff(names(df_raw), meta_cols)

plaus_cols   <- grep("Wie plausibel ist die Antwort", item_cols, value = TRUE)
quality_cols <- setdiff(item_cols, plaus_cols)

df_num <- df_raw |>
  mutate(
    across(all_of(quality_cols), ~ likert_map[.x]),
    across(all_of(plaus_cols),   ~ plaus_map[.x])
  )

# =====================================================================
# 2. Items Fragen und Dimensionen zuordnen
# =====================================================================
n_fragen <- 8
n_items_pro_frage <- length(item_cols) / n_fragen

frage_nr <- rep(1:n_fragen, each = n_items_pro_frage)

item_info <- data.frame(
  Spalte   = item_cols,
  Frage_Nr = frage_nr
)

item_info <- item_info |>
  mutate(
    Dimension = case_when(
      grepl("relevanten Informationen aus dem Kontext", Spalte) ~ "Kontextverständnis",
      grepl("korrektes Verständnis des Kontexts", Spalte)       ~ "Kontextverständnis",
      grepl("statt allgemeinem Wissen", Spalte)                 ~ "Kontextverständnis",
      
      grepl("widersprechen sich nicht", Spalte)                 ~ "Kohärenz",
      grepl("Schlussfolgerungen in der Antwort", Spalte)        ~ "Kohärenz",
      grepl("Gedankenführung der Antwort", Spalte)              ~ "Kohärenz",
      
      grepl("Detailtiefe der Antwort", Spalte)                  ~ "Angemessenheit",
      grepl("adressiert alle wesentlichen Aspekte", Spalte)     ~ "Angemessenheit",
      
      grepl("Wie plausibel ist die Antwort", Spalte)            ~ "Gesamtplausibilität",
      TRUE ~ NA_character_
    )
  )

# =====================================================================
# 3. Long-Format bauen
# =====================================================================
df_long <- df_num |>
  mutate(Episode = row_number()) |>
  pivot_longer(
    cols      = all_of(item_cols),
    names_to  = "Spalte",
    values_to = "Score"
  ) |>
  left_join(item_info, by = "Spalte")

# =====================================================================
# 4. Aggregation ins LLM-Format
# =====================================================================
df_human <- df_long |>
  group_by(Episode, Frage_Nr, Dimension) |>
  summarise(Score = mean(Score, na.rm = TRUE), .groups = "drop") |>
  pivot_wider(
    names_from  = Dimension,
    values_from = Score
  ) |>
  rename(
    Kontextverständnis_Score  = Kontextverständnis,
    Kohärenz_Score            = Kohärenz,
    Angemessenheit_Score      = Angemessenheit,
    Gesamtplausibilität_Score = Gesamtplausibilität
  ) |>
  arrange(Episode, Frage_Nr)

View(df_human)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: lpSolve



Keine Zeilen werden weggelassen.



ID,Startzeit,Fertigstellungszeit,E-Mail,Name,Zeitpunkt der letzten Änderung,Die Antwort erfasst die relevanten Informationen aus dem Kontext.,Die Antwort zeigt ein korrektes Verständnis des Kontexts.,Die Antwort nutzt relevante Informationen aus dem gegebenen Kontext zur Beantwortung der Frage (statt allgemeinem Wissen).,Die Aussagen in der Antwort widersprechen sich nicht.,⋯,"Wie plausibel ist die Antwort, basierend auf dem gegebenen Kontext?7",Die Antwort erfasst die relevanten Informationen aus dem Kontext.8,Die Antwort zeigt ein korrektes Verständnis des Kontexts.8,Die Antwort nutzt relevante Informationen aus dem gegebenen Kontext zur Beantwortung der Frage (statt allgemeinem Wissen).7,Die Aussagen in der Antwort widersprechen sich nicht.8,Die Schlussfolgerungen in der Antwort ergeben sich logisch aus den Gründen/Prämissen.8,Die Gedankenführung der Antwort ist klar und gut zu folgen.9,"Die Detailtiefe der Antwort ist passend zur Fragestellung – nicht zu oberflächlich, nicht zu ausschweifend.8",Die Antwort adressiert alle wesentlichen Aspekte der Frage – es fehlen keine wichtigen Informationen.8,"Wie plausibel ist die Antwort, basierend auf dem gegebenen Kontext?8"
<dbl>,<dttm>,<dttm>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
4,2025-12-09 18:32:40,2025-12-09 19:36:22,anonymous,NA,NA,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft überhaupt nicht zu,Trifft eher zu,⋯,"Plausibel – Könnte zutreffen, ist aber nicht der Normalfall",Trifft teilweise zu,Trifft teilweise zu,Trifft eher nicht zu,Trifft überhaupt nicht zu,Trifft teilweise zu,Trifft eher nicht zu,Trifft teilweise zu,Trifft teilweise zu,"Plausibel – Könnte zutreffen, ist aber nicht der Normalfall"
5,2025-12-11 10:43:43,2025-12-11 10:53:23,anonymous,NA,NA,Trifft eher zu,Trifft eher zu,Trifft eher zu,Trifft eher zu,⋯,"Plausibel – Könnte zutreffen, ist aber nicht der Normalfall",Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Wahrscheinlich – Klingt vernünftig und entspricht der Erwartung
6,2025-12-14 20:33:59,2025-12-15 22:14:01,anonymous,NA,NA,Trifft teilweise zu,Trifft eher zu,Trifft eher nicht zu,Trifft voll und ganz zu,⋯,Wahrscheinlich – Klingt vernünftig und entspricht der Erwartung,Trifft teilweise zu,Trifft eher zu,Trifft eher nicht zu,Trifft eher zu,Trifft teilweise zu,Trifft teilweise zu,Trifft eher zu,Trifft eher zu,"Plausibel – Könnte zutreffen, ist aber nicht der Normalfall"
7,2025-12-17 14:35:27,2025-12-17 14:55:00,anonymous,NA,NA,Trifft voll und ganz zu,Trifft teilweise zu,Trifft überhaupt nicht zu,Trifft voll und ganz zu,⋯,Sehr wahrscheinlich – Sehr überzeugend; so würde ein Experte antworten,Trifft teilweise zu,Trifft teilweise zu,Trifft teilweise zu,Trifft eher zu,Trifft eher zu,Trifft eher zu,Trifft eher zu,Trifft eher zu,"Plausibel – Könnte zutreffen, ist aber nicht der Normalfall"
8,2025-12-17 14:44:04,2025-12-17 15:10:55,anonymous,NA,NA,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft teilweise zu,Trifft voll und ganz zu,⋯,Wahrscheinlich – Klingt vernünftig und entspricht der Erwartung,Trifft eher zu,Trifft voll und ganz zu,Trifft teilweise zu,Trifft teilweise zu,Trifft teilweise zu,Trifft eher nicht zu,Trifft eher zu,Trifft teilweise zu,"Technisch möglich – Theoretisch denkbar, aber extrem unwahrscheinlich"
9,2025-12-17 16:01:47,2025-12-17 16:19:21,anonymous,NA,NA,Trifft eher nicht zu,Kann ich nicht beantworten,Trifft überhaupt nicht zu,Trifft voll und ganz zu,⋯,Wahrscheinlich – Klingt vernünftig und entspricht der Erwartung,Kann ich nicht beantworten,Kann ich nicht beantworten,Trifft eher nicht zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft eher zu,Trifft teilweise zu,"Plausibel – Könnte zutreffen, ist aber nicht der Normalfall"
10,2025-12-17 17:03:13,2025-12-17 17:52:34,anonymous,NA,NA,Trifft voll 

Episode,Frage_Nr,Angemessenheit_Score,Gesamtplausibilität_Score,Kohärenz_Score,Kontextverständnis_Score
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4.0,4,4.333333,3.666667
1,2,1.0,3,1.666667,2.000000
1,3,4.0,4,5.000000,4.666667
1,4,5.0,4,4.250000,4.500000
1,5,4.0,2,2.333333,3.666667
1,6,4.0,2,2.000000,3.333333
1,7,4.0,3,3.000000,3.000000
1,8,3.0,3,2.000000,2.666667
2,1,5.0,4,4.000000,4.000000


# =====================================================================
# 0. Setup: Pakete und Daten (LLM AS A JUDGE)
# =====================================================================


In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(reshape2)
library(knitr)
library(irr)
setwd("/Users/joern.ahlert/Desktop/Bachelorarbeit")

df_llm <- read.csv(file.path("technische Umsetzung", "LLM as a judge",
                         "llm_evaluation_results_100_episodes.csv"))
                         
View(df_llm)

Episode,Frage_Nr,Frage,Kontextverständnis_Score,Kohärenz_Score,Angemessenheit_Score,Gesamtplausibilität_Score
<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Welche Vorteile könnte der Kaffernbüffel aus Leben...,2.93,5,5.00,3.00
1,2,Warum kann der Kaffernbüffel auch in relativ trock...,4.69,5,4.62,3.00
1,3,Wie verändert sich das Verhalten des Kaffernbüffel...,4.80,5,5.00,3.17
1,4,Weshalb sind Bullen nicht Teil des inneren Sozialg...,4.54,5,5.00,3.29
1,5,Welche Rolle spielt die Position eines Tieres inne...,4.60,5,5.00,3.12
1,6,Aus welchen Gründen könnten weibliche Kaffernbüffe...,4.53,5,5.00,3.16
1,7,Welche Auswirkungen kann die Stellung eines Tieres...,5.00,5,5.00,4.72
1,8,Weshalb sind große Herden in bestimmten Jahreszeit...,4.63,5,5.00,3.46
2,1,Welche Vorteile könnte der Kaffernbüffel aus Leben...,2.99,5,5.00,3.00


# =====================================================================
# 1. Deskriptive Statistik – dimensionsspezifisch
# =====================================================================


In [3]:
# =====================================================================
# 5. Daten aggregieren (Mittelwerte pro Frage bilden)
# =====================================================================

# A) MENSCHLICHE DATEN (df_human)
# Gruppieren nach Frage_Nr -> Mittelwert über alle menschlichen Rater
df_human_agg <- df_human %>%
  group_by(Frage_Nr) %>%
  summarise(
    Kontext_Human    = mean(Kontextverständnis_Score, na.rm = TRUE),
    Kohaerenz_Human  = mean(Kohärenz_Score, na.rm = TRUE),
    Angemessen_Human = mean(Angemessenheit_Score, na.rm = TRUE),
    Gesamt_Human     = mean(Gesamtplausibilität_Score, na.rm = TRUE)
  )

# B) LLM DATEN (df_llm)
# Gruppieren nach Frage_Nr -> Mittelwert über alle LLM-Iterationen
df_llm_agg <- df_llm %>%
  group_by(Frage_Nr) %>%
  summarise(
    Kontext_LLM    = mean(Kontextverständnis_Score, na.rm = TRUE),
    Kohaerenz_LLM  = mean(Kohärenz_Score, na.rm = TRUE),
    Angemessen_LLM = mean(Angemessenheit_Score, na.rm = TRUE),
    Gesamt_LLM     = mean(Gesamtplausibilität_Score, na.rm = TRUE)
  )

# =====================================================================
# 6. Zusammenfügen & Korrelation berechnen
# =====================================================================

# Join über Frage_Nr (ergibt Tabelle mit 8 Zeilen)
df_compare <- left_join(df_human_agg, df_llm_agg, by = "Frage_Nr")

# Hilfsfunktion für Spearman & Kendall
calc_corr <- function(data, col_human, col_llm, label) {
  s <- cor.test(data[[col_human]], data[[col_llm]], method = "spearman")
  k <- cor.test(data[[col_human]], data[[col_llm]], method = "kendall")
  
  data.frame(
    Dimension = label,
    Spearman_Rho = round(s$estimate, 3),
    Spearman_p   = format.pval(s$p.value, digits = 3, eps = 0.001),
    Kendall_Tau  = round(k$estimate, 3),
    Kendall_p    = format.pval(k$p.value, digits = 3, eps = 0.001),
    N_Items      = nrow(data),
    row.names    = NULL
  )
}

# Berechnung für alle 4 Dimensionen
results <- bind_rows(
  calc_corr(df_compare, "Kontext_Human",    "Kontext_LLM",    "Kontextverständnis"),
  calc_corr(df_compare, "Kohaerenz_Human",  "Kohaerenz_LLM",  "Kohärenz"),
  calc_corr(df_compare, "Angemessen_Human", "Angemessen_LLM", "Angemessenheit"),
  calc_corr(df_compare, "Gesamt_Human",     "Gesamt_LLM",     "Gesamtplausibilität")
)

# =====================================================================
# 7. Ausgabe & LaTeX
# =====================================================================

print(results)

# LaTeX-Tabelle erstellen
print(xtable(results, caption = "Korrelation zwischen menschlicher Bewertung und LLM-Judge (aggregiert auf Frage-Ebene)"), 
      include.rownames = FALSE)


Warning message in cor.test.default(data[[col_human]], data[[col_llm]], method = "spearman"):
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data[[col_human]], data[[col_llm]], method = "kendall"):
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data[[col_human]], data[[col_llm]], method = "spearman"):
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data[[col_human]], data[[col_llm]], method = "kendall"):
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data[[col_human]], data[[col_llm]], method = "spearman"):
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(data[[col_human]], data[[col_llm]], method = "kendall"):
“Cannot compute exact p-value with ties”


            Dimension Spearman_Rho Spearman_p Kendall_Tau Kendall_p N_Items
1  Kontextverständnis        0.503      0.204       0.400      0.17       8
2            Kohärenz       -0.439      0.276      -0.416     0.161       8
3      Angemessenheit        0.908    0.00181       0.847   0.00477       8
4 Gesamtplausibilität        0.524      0.197       0.357     0.275       8
% latex table generated in R 4.5.2 by xtable 1.8-4 package
% Mon Dec 22 13:00:36 2025
\begin{table}[ht]
\centering
\begin{tabular}{lrlrlr}
  \hline
Dimension & Spearman\_Rho & Spearman\_p & Kendall\_Tau & Kendall\_p & N\_Items \\ 
  \hline
Kontextverständnis & 0.50 & 0.204 & 0.40 & 0.17 &   8 \\ 
  Kohärenz & -0.44 & 0.276 & -0.42 & 0.161 &   8 \\ 
  Angemessenheit & 0.91 & 0.00181 & 0.85 & 0.00477 &   8 \\ 
  Gesamtplausibilität & 0.52 & 0.197 & 0.36 & 0.275 &   8 \\ 
   \hline
\end{tabular}
\caption{Korrelation zwischen menschlicher Bewertung und LLM-Judge (aggregiert auf Frage-Ebene)} 
\end{table}


# Gewichteete Dimensionen Kandall tau und Pearson

In [4]:
# =====================================================================
# 1. Gewichtete Scores berechnen (in den aggregierten Dataframes)
# =====================================================================

# Gewichtung definieren
w_kontext <- 0.3
w_kohaerenz <- 0.4
w_angemessen <- 0.3

# A) MENSCHLICHE DATEN: Gewichteten Score hinzufügen
df_human_agg <- df_human_agg %>%
  mutate(
    Gewichtet_Human = (Kontext_Human * w_kontext) + 
                      (Kohaerenz_Human * w_kohaerenz) + 
                      (Angemessen_Human * w_angemessen)
  )

# B) LLM DATEN: Gewichteten Score hinzufügen
df_llm_agg <- df_llm_agg %>%
  mutate(
    Gewichtet_LLM = (Kontext_LLM * w_kontext) + 
                    (Kohaerenz_LLM * w_kohaerenz) + 
                    (Angemessen_LLM * w_angemessen)
  )

# =====================================================================
# 2. Zusammenfügen & Korrelieren
# =====================================================================

df_compare_weighted <- left_join(df_human_agg, df_llm_agg, by = "Frage_Nr")

# Korrelation berechnen: Gewichtet (Mensch) vs. Gewichtet (LLM)
res_weighted <- calc_corr(df_compare_weighted, "Gewichtet_Human", "Gewichtet_LLM", "Plausibilität (Gewichtet)")

# Korrelation für Gesamtplausibilität (falls noch nicht berechnet)
res_gesamt <- calc_corr(df_compare_weighted, "Gesamt_Human", "Gesamt_LLM", "Gesamtplausibilität")

# =====================================================================
# 3. Ausgabe
# =====================================================================

print(res_gesamt)
print(res_weighted)

# LaTeX Code für beide Zeilen
results_final <- bind_rows(res_gesamt, res_weighted)
print(xtable(results_final, caption = "Korrelation zwischen menschlichen und automatischen Bewertungen"), 
      include.rownames = FALSE)


            Dimension Spearman_Rho Spearman_p Kendall_Tau Kendall_p N_Items
1 Gesamtplausibilität        0.524      0.197       0.357     0.275       8
                  Dimension Spearman_Rho Spearman_p Kendall_Tau Kendall_p
1 Plausibilität (Gewichtet)        0.119      0.793       0.143      0.72
  N_Items
1       8
% latex table generated in R 4.5.2 by xtable 1.8-4 package
% Mon Dec 22 13:00:36 2025
\begin{table}[ht]
\centering
\begin{tabular}{lrlrlr}
  \hline
Dimension & Spearman\_Rho & Spearman\_p & Kendall\_Tau & Kendall\_p & N\_Items \\ 
  \hline
Gesamtplausibilität & 0.52 & 0.197 & 0.36 & 0.275 &   8 \\ 
  Plausibilität (Gewichtet) & 0.12 & 0.793 & 0.14 & 0.72 &   8 \\ 
   \hline
\end{tabular}
\caption{Korrelation zwischen menschlichen und automatischen Bewertungen} 
\end{table}
